# Read list of news items aggregates with exactly one news item entry

In [9]:
import json
with open("../data/incremental_clustering/test_to_cluster_v2.json") as f:
    news_item_list = json.load(f)


In [10]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Using inital clustering, input is a list of news items aggregates where each has exactly one news item entry

In [4]:
from story_clustering.clustering import initial_clustering

/home/andreselm/storage/envs/keybert/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andreselm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Story Clustering Setup Complete.


In [12]:
# use initial clustering over unclustered news items 
initial_clusters = initial_clustering(news_item_list)
print(initial_clusters)

/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values.loc[:, ("Similarity")] = 1
/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values.loc[:, ("Similarity")] = 1
/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

{'event_clusters': [[12443], [11931, 12063], [11803], [11907], [11533], [11716]]}


# Printing news items titles belonging to the same event

In [15]:
def group_aggregate(aggregate_ids: list[int], news_item_list):
        first_id = aggregate_ids.pop(0)
        first_aggregate = [dic for dic in news_item_list if dic['id'] == first_id][0]
        if not first_aggregate:
            print("error not_found")
        processed_aggregates = [first_aggregate]
        for item in aggregate_ids:
            aggregate = [dic for dic in news_item_list if dic['id'] == item ][0]
            if not aggregate:
                continue
            #first_aggregate['tags'] = list({tag['name']: tag for tag in first_aggregate['tags'].update(aggregate['tags'])}.values())
            first_aggregate['tags'] = first_aggregate['tags'] | aggregate['tags']
            for news_item in aggregate['news_items'][:]:
                first_aggregate['news_items'].append(news_item)
                first_aggregate['relevance'] += 1
                aggregate['news_items'].remove(news_item)
            #processed_aggregates.extend(aggregate)
        return processed_aggregates

In [ ]:
def create_initial_clusters_aggregates(news_item_list):
    
    initial_clusters = initial_clustering(news_item_list)
    results = []
    for aggregate_ids in initial_clusters["event_clusters"]:
        results.extend(group_aggregate(aggregate_ids, news_item_list))
    #results = [group_aggregate(aggregate_ids, news_item_list) for aggregate_ids in clusters]
    return results

In [16]:
def print_titles_initial_clusters(initial_clusters,news_item_list):
    for event_docs_list in initial_clusters["event_clusters"]:
        print("---- docs belonging to the same event ----")
        for doc_id in event_docs_list:
            title = [d["news_item_data"]["title"] for ni in news_item_list for d in ni["news_items"] if ni["id"] == doc_id ]
            print(title[0])
        print("-------------")

In [17]:
print_titles_initial_clusters(initial_clusters,news_item_list)

---- docs belonging to the same event ----
Slovenian Electrical Utility HSE Suffers Ransomware Attack
-------------
---- docs belonging to the same event ----
Hamas-Linked APT Wields New SysJoker Backdoor Against Israel
Hamas-Linked APT Wields New SysJoker Backdoor Against Israel
-------------
---- docs belonging to the same event ----
Wie man private Pkw am Land ersetzt
-------------
---- docs belonging to the same event ----
New Rust-based SysJoker backdoor linked to Hamas hackers
-------------
---- docs belonging to the same event ----
Hamas-Linked Cyberattacks Using Rust-Powered SysJoker Backdoor Against Israel
-------------
---- docs belonging to the same event ----
DSA-5566-1 thunderbird - security update
-------------


# Cluster news items aggregates against existing clusters

In [18]:
with open("../data/incremental_clustering/test_already_clustered_v2.json") as f:
    clustered_news_items = json.load(f)

In [19]:
from story_clustering.clustering import incremental_clustering_v2

In [20]:
updated_clusters = incremental_clustering_v2(news_item_list, clustered_news_items)
print(updated_clusters)

/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values.loc[:, ("Similarity")] = 1
/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values.loc[:, ("Similarity")] = 1
/home/andreselm/awake-tools/clustering-task/story-clustering/story_clustering/nlp_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

{'event_clusters': [[12443, 12443], [12139], [12292], [12395], [11662], [11475], [11730], [11446], [11153], [11803, 11803], [11744], [12220], [11852], [11931, 12063, 11907, 11533], [12023], [11358], [11716, 11716], [11331], [12008], [11455]]}


In [24]:

def print_titles_only(news_item_list):
    for item in news_item_list:
        groupid = item["id"]
        print(f"---- Group id {groupid} ----")
        for ni in item["news_items"]:
            id = ni["id"]
            title = ni["news_item_data"]["title"]
            print(f"Id: {id}, title: {title}")
        print("-------------")

def print_titles_incremental_clusters(incremental_clusters,news_item_list, already_clustered_news_item_list):
    results = []
    all_items = []
    for item in already_clustered_news_item_list:
        all_items.append(item)
    for item in news_item_list:
        all_items.append(item)
    for aggregate_ids in incremental_clusters["event_clusters"]:
        results.extend(group_aggregate(aggregate_ids, all_items))
    print_titles_only(results)

In [25]:
print_titles_incremental_clusters(updated_clusters,news_item_list,clustered_news_items)

---- Group id 12443 ----
Id: 11491, title: Slovenia's largest power provider HSE hit by ransomware attack
Id: 11787, title: Slovenian power company hit by ransomware
-------------
---- Group id 12139 ----
Id: 11726, title: How to Read the Israeli “Kidnapped” Posters
-------------
---- Group id 12292 ----
Id: 11879, title: Hackers Spent 2+ Years Looting Secrets of Chipmaker NXP Before Being Detected
Id: 11728, title: Hackers spent 2+ years looting secrets of chipmaker NXP before being detected
Id: 11371, title: Niederländischer Chipdesigner NXP 2017-2020 durch mutmaßlich chinesische Hacker infiltriert
Id: 11266, title: Chinesische Cyberkriminelle hatten jahrelang Zugriff auf Chiphersteller NXP
Id: 11265, title: Chinesische Cyberkriminelle hatten jahrelang Zugriff auf Chiphersteller NXP
-------------
---- Group id 12395 ----
Id: 11982, title: Iranian Hackers Exploit PLCs in Attack on Water Authority in U.S.
Id: 12035, title: CISA Warns of Unitronics PLC Exploitation Following Water Utili